# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.core.model import Model
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

experiment_name = 'capstoneHyper'
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A52NFTU9N to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-143194
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-143194


In [4]:
# Get the data of Kaggle Titanic Dataset
key = "titanic-dataset"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    titanicData = 'https://raw.githubusercontent.com/kkech/UdacityProj3_MS/master/train.csv'
    dataset = Dataset.Tabular.from_delimited_files(titanicData)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
compute_cluster_name = "titaniccluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
# Create dependencies file for the train script
import os
import shutil

project_folder = './sklearnTitanic'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)
shutil.copy('config.json', project_folder)
            
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [9]:
# Create an early termination policy. 
early_termination_policy = BanditPolicy(slack_factor = 0.1)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    '--C': uniform(0.001, 1.0),
    '--max_iter': choice(1000, 10000, 100000)
} )

# Create your estimator and hyperdrive config
src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100)

In [10]:
#TODO: Submit your experiment
hdr = exp.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
hdr.wait_for_completion(show_output = True)

RunId: HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3
Web View: https://ml.azure.com/runs/HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-143194/workspaces/quick-starts-ws-143194&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-19T23:17:06.252494][API][INFO]Experiment created<END>\n""<START>[2021-04-19T23:17:06.923005][GENERATOR][INFO]Trying to sample '100' jobs from the hyperparameter space<END>\n""<START>[2021-04-19T23:17:07.674301][GENERATOR][INFO]Successfully sampled '100' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3
Web View: https://ml.azure.com/runs/HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-143194/workspaces/quick-starts-ws-143194&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3',
 'target': 'titaniccluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T23:17:06.048645Z',
 'endTimeUtc': '2021-04-19T23:33:15.446069Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6a6c7381-0ad5-46d5-93dd-7b942d3dd3af',
  'score': '0.7443946188340808',
  'best_child_run_id': 'HD_b2ce29b2-b0c9-4822-a835-d882054d3dc3_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg143194.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b2ce29b2-b0c9-4822-a835-d882054d

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
#TODO: Save the best model
import joblib

best_run = hdr.get_best_run_by_primary_metric()

best_run = hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())
print(best_run.get_metrics())

model = best_run.register_model(model_name='best-sklearn-model',
                                model_path='outputs/model.joblib',
                                model_framework=Model.Framework.SCIKITLEARN)

['--C', '0.1275319312749521', '--max_iter', '100000']
['azureml-logs/20_image_build_log.txt', 'azureml-logs/55_azureml-execution-tvmps_a88f405f0d22998215037ffb2e84131a69b5755baef397ce6e592d42ec4293d5_d.txt', 'azureml-logs/65_job_prep-tvmps_a88f405f0d22998215037ffb2e84131a69b5755baef397ce6e592d42ec4293d5_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a88f405f0d22998215037ffb2e84131a69b5755baef397ce6e592d42ec4293d5_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/104_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
{'Regularization Strength': 0.1275319312749521, 'Max iterations': 100000, 'Accuracy': 0.7443946188340808}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#BEST MODEL IS AUTOML - SO I DID NOT DEPLOY THIS.

service_name = 'hyperdrive-service'
service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print(service.state)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Fare": 13.0,
            "Q": 0,
            "S": 1,
            "male": 1
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Fare": 512.3292,
            "Q": 0,
            "S": 0,
            "male": 1
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

output = service.run(input_data)
print(output)

In [ ]:
service.delete()
print(service.state)